In [2]:
# Libraries
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scipy
!pip install sklearn
!pip install seaborn


Could not find platform independent libraries <prefix>


Could not find platform independent libraries <prefix>


  Using cached matplotlib-3.10.3-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.2-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.5-cp313-cp313-win_amd64.whl.metadata (109 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.3-cp313-cp313-win_amd64.whl (8.1 MB)
Using cached contourpy-1.3.2-cp313-cp313-win_amd64.whl (223 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.58.5-cp313-cp313-win_amd64.whl (2.2 MB)
Using cached kiwisolver-1.4.8-cp313-cp313-win_amd64.whl (71 kB)
Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl (7.0 MB)
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)

   ----- ---------------------------------- 1/7 [pillow]
   ----

Could not find platform independent libraries <prefix>


  Using cached scipy-1.16.0-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.16.0-cp313-cp313-win_amd64.whl (38.4 MB)


Could not find platform independent libraries <prefix>


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


Could not find platform independent libraries <prefix>
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      Could not find platform independent libraries <prefix>
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=T

Could not find platform independent libraries <prefix>


In [4]:
import numpy as np
import pandas as pd
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib.ticker import MultipleLocator
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import math

In [12]:
# Load and prepare data
def prepare_data(train_file='data/trainEgEf.txt', test_file='data/testEgEf.txt', pca_components=5):
    # Load CSVs
    df_train = pd.read_csv(train_file).drop(columns=["Ef"])
    df_test = pd.read_csv(test_file).drop(columns=["Ef"])

    # Separate features and targets
    X_train = df_train.drop(columns=["Eg"])
    y_train = df_train["Eg"].astype(float).values
    X_test = df_test.drop(columns=["Eg"])
    y_test = df_test["Eg"].astype(float).values

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)  # FIXED: no fit on test

    # SVR pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVR())
    ])

    param_grid = {
        'svm__C': [100],
        'svm__gamma': ['auto'],
        'svm__kernel': ['rbf'],
        'svm__epsilon': [0.001]
    }

    grid_search = GridSearchCV(pipeline, param_grid, cv=5)
    grid_search.fit(X_train_scaled, y_train)
    y_pred = grid_search.predict(X_test_scaled)
    prediction_errors = np.abs(y_pred - y_test)

    # PCA transformation on features only
    pca = PCA(n_components=pca_components)
    X_test_pca = pca.fit_transform(X_test_scaled)

    # Stack: PCA, error, actual, predicted
    result = np.hstack([
        X_test_pca,
        prediction_errors.reshape(-1, 1),
        y_test.reshape(-1, 1),
        y_pred.reshape(-1, 1)
    ])

    return result

In [13]:
# Point-in-hull test
def point_in_hull(point, hull, tol=1e-12):
    return all(np.dot(eq[:-1], point) + eq[-1] <= tol for eq in hull.equations)

In [14]:
# Hull optimization
def optimize_hull(data, max_iter=2000):
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    history_cost, history_error = [], []
    working_train = train_data.copy()

    for iteration in range(max_iter):
        if iteration == 0 or len(working_train) < 25:
            hull = ConvexHull(working_train[:, :5])
        else:
            hull = ConvexHull(working_train[:, :5])

        inside_points = []
        inside_errors = []
        cost_components = []

        for row in train_data:
            point, err = row[:5], row[5]
            if point_in_hull(point, hull):
                inside_errors.append(err)
                cost_components.append(0.0 if err <= np.percentile(train_data[:, 5], 5) else math.exp(3.0 * err))
                inside_points.append(row)

        if not inside_points:
            continue

        inside_points = np.array(inside_points)
        mean_error = np.mean(inside_errors)
        cost = -len(inside_points) + np.sum(cost_components)

        history_cost.append(cost)
        history_error.append(mean_error)

        # Update strategy
        if len(history_cost) > 1 and cost < history_cost[-2] and mean_error < history_error[-2]:
            working_train = inside_points
        elif mean_error < 0.022:
            final_train = inside_points
        else:
            n_add = 6 if cost < (1 / 3) * history_cost[0] else 12
            new_candidates = train_data[np.random.choice(len(train_data), n_add, replace=False)]
            working_train = np.vstack([new_candidates, inside_points])

        if mean_error < 0.65 * history_error[0] and len(inside_points) > 50:
            return working_train, test_data, ConvexHull(inside_points[:, :5]), history_cost, history_error

    return working_train, test_data, hull, history_cost, history_error

In [15]:
# Plotting
def plot_results(train_data, test_data, hull):
    # Distance to hull
    distances = [np.max(np.dot(hull.equations[:, :-1], p) + hull.equations[:, -1]) for p in test_data[:, :5]]

    # Plot 1: Distance vs error
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(distances, test_data[:, 5], 'bo')
    ax.set(xlabel="Convex Hull Distance", ylabel="Prediction Error (eV)", facecolor='w')
    ax.xaxis.set_major_locator(MultipleLocator(0.01))
    ax.yaxis.set_major_locator(MultipleLocator(0.1))
    ax.grid(True)
    for spine in ax.spines.values():
        spine.set_color('black')
    plt.xlim(-0.017, 0.05)
    plt.savefig('HullDistance_vs_Error.svg')
    plt.close()

    # Plot 2: KDE Error distributions
    err_bins = {
        'inside': [test_data[i, 5] for i in range(len(distances)) if distances[i] < 0.2 * min(distances)],
        'edge': [test_data[i, 5] for i in range(len(distances)) if 0.2 * min(distances) <= distances[i] <= 0.2 * max(distances)],
        'outside': [test_data[i, 5] for i in range(len(distances)) if distances[i] > 0.2 * max(distances)]
    }

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.kdeplot(err_bins['inside'], color='red', label='Inside Hull', lw=3)
    sns.kdeplot(err_bins['edge'], color='blue', label='Edge of Hull', lw=3)
    sns.kdeplot(err_bins['outside'], color='green', label='Outside Hull', lw=3)

    plt.axvline(x=np.percentile(train_data[:, 5], 5), color='b', label='Error Cutoff')
    plt.xlim(0.0, 1.0)
    plt.title('Error Distribution by Hull Distance', fontsize=18)
    plt.xlabel('Prediction Error (eV)', fontsize=18)
    plt.ylabel('Density', fontsize=18)
    plt.legend()
    for spine in ax.spines.values():
        spine.set_color('black')
    plt.savefig('Hull_Error_Distribution.svg')
    plt.close()



In [16]:
# Entry point for Jupyter
def main():
    data = prepare_data()
    train_data, test_data, final_hull, cost_history, error_history = optimize_hull(data)
    plot_results(train_data, test_data, final_hull)
    return train_data, test_data, final_hull, cost_history, error_history


Suggested Additional Visualizations

In [17]:
from mpl_toolkits.mplot3d import Axes3D

def plot_pca_3d(data):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    sc = ax.scatter(data[:, 0], data[:, 1], data[:, 2],
                    c=data[:, 5], cmap='viridis', s=30, alpha=0.8)

    ax.set_xlabel('PCA 1')
    ax.set_ylabel('PCA 2')
    ax.set_zlabel('PCA 3')
    fig.colorbar(sc, ax=ax, label='Prediction Error (eV)')
    plt.title('3D PCA Space Colored by Error')
    plt.tight_layout()
    plt.savefig('PCA_3D_Error_Scatter.svg')
    plt.close()


In [18]:
def plot_hull_inclusion(data, hull):
    inclusion = np.array([point_in_hull(p, hull) for p in data[:, :5]])
    inclusion_labels = np.where(inclusion, 'Inside Hull', 'Outside Hull')

    df_plot = pd.DataFrame({
        'PCA_1': data[:, 0],
        'PCA_2': data[:, 1],
        'Error': data[:, 5],
        'Inclusion': inclusion_labels
    })

    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df_plot, x='PCA_1', y='PCA_2',
                    hue='Inclusion', style='Inclusion', size='Error',
                    palette={'Inside Hull': 'green', 'Outside Hull': 'red'},
                    alpha=0.7)

    plt.title('Hull Inclusion by Error')
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.legend()
    plt.savefig('Hull_Inclusion_Error_Scatter.svg')
    plt.close()


In [22]:
def plot_parity(data):
    true_Eg = data[:, 6]
    pred_Eg = data[:, 7]

    plt.figure(figsize=(8, 8))
    plt.plot([min(true_Eg), max(true_Eg)], [min(true_Eg), max(true_Eg)], 'k--', label='Perfect Prediction')
    plt.scatter(true_Eg, pred_Eg, c=data[:, 5], cmap='plasma', s=30, edgecolors='k', alpha=0.7)
    plt.xlabel('True Eg (eV)')
    plt.ylabel('Predicted Eg (eV)')
    plt.title('Parity Plot of Eg Prediction')
    plt.colorbar(label='Absolute Error')
    plt.legend()
    plt.grid(True)
    plt.savefig('Eg_ParityPlot.svg')
    plt.show()
    plt.close()


In [21]:
def main():
    data = prepare_data()
    train_data, test_data, final_hull, cost_history, error_history = optimize_hull(data)

    # Original diagnostics
    plot_results(train_data, test_data, final_hull)

    # Extended visualizations
    plot_pca_3d(train_data)
    plot_hull_inclusion(train_data, final_hull)
    plot_parity(train_data)
